In [5]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from time import sleep
from gestione_input import leggi_float


# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2' 
# ]

url_base = 'https://www.subito.it/annunci-italia/vendita/usato/'

def scrape_page(page):
    query_str = {
        'q' : 'spectrum+sinclair',
        'o' : page
    }


    result = requests.get(url_base, params=query_str)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # controllare se non è una pagina vuota
        no_result = soup.select('div[class*=ErrorLayout_container__]')
        if no_result:
            raise requests.RequestException('Nessun risultato al momento')

        products = soup.select('div.items__item.item-card')
        # pprint(products)


        items= []

        for adv in products:
            data = {
            'name' : adv.select('h2[class*=ItemTitle-module_item-title__]'),
            'price' : adv.select('p.price'),
            'town' : adv.select('span[class*=index-module_town__]'),
            'city' : adv.select('span.city'),
            'shipping' : adv.select('span.shipping-badge'), 
            'sold' : adv.select('span.item-sold-badge'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else: 
                    data[key] = None
                
                # oppure si può anche scrivere così
                #data[key] = value[0].get_text() if value else None

            data.update({
                'price':float(data['price'].replace('Spedizione disponibile', '')\
                        .replace('\xa0€', ' ')\
                        .replace('Venduto', '')) if data['price'] else None,
                'url': adv.select('a.link')[0]['href'],
            })

            items.append(data)

    return items 
    # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', '')

current_page = 1
all_data = []
max_page = leggi_float('Quante pagine vuoi analizzare?'
                 'Premi invio per tutte le pagine disponibili', None, None)

while True: 
    if not max_page or current_page <= max_page:
        try:
            items = scrape_page(current_page)
            
        except requests.RequestException as err:
            print(err)
            break

        all_data+= items
        current_page += 1
        sleep(1) 

    else:
        break

    # print(items)
    # print(current_page)

pprint(all_data)

UnboundLocalError: local variable 'risposta' referenced before assignment